In [ ]:
import pandas as pd
import psycopg2

import os
import sys
sys.path.append("../")
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Criando as variáveis para conexão das bases
database = os.getenv('DATABASE')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
port = int(os.getenv('PORT'))
host = os.getenv('HOST')

<div style="text-align:center">
    <h1 style="font-size:40px"><b> Inserção de Dados no Postgres </b></h1>
</div>

### Criando as tabelas no Postgres

```sql
CREATE TABLE movies (
  id SERIAL PRIMARY KEY,
  title VARCHAR(255) NOT NULL,
  year INTEGER NOT NULL,
  duration INTEGER NOT NULL
);```sql

```sql
CREATE TABLE genres (
  id SERIAL PRIMARY KEY,
  name VARCHAR(255) NOT NULL
);```sql

```sql
CREATE TABLE actors (
  id SERIAL PRIMARY KEY,
  name VARCHAR(255) NOT NULL
);```sql

```sql
CREATE TABLE roles (
  id SERIAL PRIMARY KEY,
  movie_id INTEGER NOT NULL REFERENCES movies(id) ON DELETE CASCADE,
  actor_id INTEGER NOT NULL REFERENCES actors(id) ON DELETE CASCADE,
  character VARCHAR(255) NOT NULL
);```sql

```sql
CREATE TABLE ratings (
  id SERIAL PRIMARY KEY,
  movie_id INTEGER NOT NULL REFERENCES movies(id) ON DELETE CASCADE,
  source VARCHAR(255) NOT NULL,
  rating FLOAT NOT NULL
);```sql

```sql
ALTER TABLE roles ADD CONSTRAINT roles_movie_id_fkey1 FOREIGN KEY (movie_id) REFERENCES movies(id) ON DELETE CASCADE;```sql

```sql
ALTER TABLE roles ADD CONSTRAINT roles_actor_id_fkey FOREIGN KEY (actor_id) REFERENCES actors(id) ON DELETE CASCADE;```sql

```sql
ALTER TABLE ratings ADD CONSTRAINT ratings_movie_id_fkey FOREIGN KEY (movie_id) REFERENCES movies(id) ON DELETE CASCADE;```sql

In [ ]:
import psycopg2

class PostgresConnector:
    def __init__(self, host, database, user, password):
        self.conn = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password
        )
        self.cur = self.conn.cursor()

    def create_table(self, table_name):
        query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id SERIAL PRIMARY KEY,
            title VARCHAR(255) UNIQUE,
            duration INTEGER,
            rating NUMERIC(3,1),
            votes INTEGER,
            year_ INTEGER
        )
        """
        self.cur.execute(query)
        self.conn.commit()

    def insert_data_from_dataframe(self, table_name, df):
        for i, row in df.iterrows():
            query = f"INSERT INTO {table_name} (title, duration, rating, votes, year_) VALUES (%s, %s, %s, %s, %s) ON CONFLICT (title) DO NOTHING"
            values = (row['title'], row['duration'], row['rating'], row['votes'], row['year'])
            print(values)
            try:
                self.cur.execute(query, values)
            except psycopg2.errors.UniqueViolation as e:
                print(f'Error: {e}')
                # ignore duplicate entries
                pass
        self.conn.commit()

    def close_connection(self):
        self.cur.close()
        self.conn.close()

In [ ]:
df_imdb_copy = pd.read_csv("../data/df_imdb_copy.csv")

In [ ]:
df_imdb_copy.head()

In [ ]:
# Instanciar a classe PostgresConnector
connector = PostgresConnector(
    host=host,
    database=database,
    user=user,
    password=password
)

# Criar tabela "movies"
connector.create_table("movies")

# Inserir dados na tabela "movies" em lotes de 1000 linhas
chunk_size = 1000
for i in range(0, len(df_imdb_copy), chunk_size):
    chunk = df_imdb_copy.iloc[i:i+chunk_size]
    connector.insert_data_from_dataframe(chunk, conn, "movies")

# Fechar a conexão
connector.close_connection()